# Moby GRPC Client

In [1]:
import sys
sys.path.append("../../python")

from neuromeka.moby import MobyClient
import time

step_ip = '192.168.214.20'
moby = MobyClient(step_ip)

## Get Moby State

### get_state
- STRING STATE
    - status : current moby state
    
      - |  status     |              description                 |
        |:-----------:|------------------------------------------|
        | SYSTEM_OFF  | None                                     |
        | SYSTEM_ON   | Initialize System and Servo On           |
        | IDLE        | Idle State, Wait for command             |
        | MOVING      | Drive by move rotation or driving        |
        | TELE_OP     | Drive by set_target_vel                  |
        | VIOLATE     | get Violation                            |
        | RECOVER_SOFT| Recover Error State                      |
    
- BOOL STATE
    - is_ready : ethercat connected and master&slaves operational
    - is_enable : all wheel servo on
    - is_moving : move(Move, Teleop) state
    - is_violation : violation

In [3]:
moby.get_moby_state()

### get_moby_error_state

[error state, num, none, none]
- error state:

                None, SW_MASTER_NOT_OP, SW_SLAVES_NOT_OP, SW_SERVO_NOT_ON,
                SW_SERVO_NOT_READY, SW_ENCODER_ABNORMAL, SW_BUMPER_DETECT

In [4]:
print(moby.get_moby_error_state())

### recover from error state

In [20]:
moby.recover()

## Get BMS Data

- GreenPyzzle : 

            'BMS status-1', 'BMS status-2',
            'Pack voltage-1', 'Pack voltage-2',
            'Battery Voltage-1', 'Battery Voltage-2',
            'Pack current1-1', 'Pack current1-2', 'Pack current2-1', 'Pack current2-2',
            'Temperature-1', 'Temperature-2', 'Temperature-3', 'Temperature-4'
- CTNS : 

            'Pack voltage-1', 'Pack current1-1
            'Is Charge', 'Is Cell OverVoltage',
            'Is Cell UnderVoltage', 'Is OverCurrent Charge',
            'Is OverCurrent Discharge', 'Is Short Circuit',
            'Is OverTemperature', 'Is Pack OverVoltage',
            'SOC', 'SOH', 'Time for Charge', 'time for Discharge'
            'Remain Capacity Ah', 'Remain Capacity Wh'
            'Temperature-(1~3)', 'Cell Voltage-(1~13)'

In [5]:
moby.get_bms()

## Get Moby's pose related

In [11]:
print("Pose [Px, Py, Pw]", [format(p, '.3f') for p in moby.get_moby_pose()])

print("Vel [Px, Py, Pw]", [format(v, '.3f') for v in moby.get_moby_vel()])

rot = moby.get_rotation_angle()
print("Rotation angle (deg)", [format(rot['fl'], '.3f'), format(rot['fr'], '.3f'), format(rot['bl'], '.3f'), format(rot['br'], '.3f')])

drv = moby.get_drive_speed()
print("Driving speed (m/s)", [format(drv['fl'], '.3f'), format(drv['fr'], '.3f'), format(drv['bl'], '.3f'), format(drv['br'], '.3f')])


In [34]:
moby.reset_moby_pose()

## Gyro data

In [12]:
print("Gyro [yaw, yaw_rate]", moby.get_gyro_data())

In [13]:
imu = moby.get_imu_data()
print(imu)

In [18]:
moby.use_gyro_for_odom(use_gyro=True)

In [30]:
moby.reset_gyro()

### UltraSonic Proximity Sensor

In [23]:
moby.get_us_data()

## Set Param

### set rotation motor zero as current pose

In [ ]:
moby.set_zero_as_current()

In [14]:
moby.get_zero()

### set rotation interpolator
0: Ramp interpolator  　　　　  1: Streaming interpolator

2: Velocity interpolator   　　　 3: Trapezoidal interpolator

In [142]:
moby.set_rotation_interpolator(val=0)

### set rotation controller type

0: HINFPID_CONTROLLER

1: SIMPLEPID_POS_CONTROLLER

2: SIMPLEPID_VEL_CONTROLLER

In [141]:
moby.set_rotation_controller_type(val=0)

### set rotation vel acc
- (vel,　acc)

In [36]:
moby.set_rotation_vel_acc(vel=1.884, acc=900.477)

### set drive interpolator on/off

In [48]:
moby.set_drive_interpolator_on_off(on=True)

### set drive acc dec
- (acc,　dec)

In [46]:
moby.set_drive_acc_dec(acc = 0.01, dec = 0.01)


### set Moby Rotation Motor Gain
index : front left:0,　 frong right:1,　 back left:2,　 back right:3

In [27]:
moby.set_rotation_gain(index=0, k=4000, kv=2000, kp=300)
moby.set_rotation_gain(index=1, k=4000, kv=2000, kp=300)
moby.set_rotation_gain(index=2, k=4000, kv=2000, kp=300)
moby.set_rotation_gain(index=3, k=4000, kv=2000, kp=300)

In [2]:
print(moby.get_rotation_gain(index=0))

## Rotation, Driving Move control

### get rotation angle, driving speed, robot velocity

In [3]:
rot = moby.get_rotation_angle()
print("Rotation angle (deg)", [format(rot["fl"], '.3f'), format(rot["fr"], '.3f'), format(rot["bl"], '.3f'), format(rot["br"], '.3f')])

drv = moby.get_drive_speed()
print("drive speed (m/s)", [format(drv["fl"], '.3f'), format(drv["fr"], '.3f'), format(drv["bl"], '.3f'), format(drv["br"], '.3f')])

body = moby.get_target_vel()
print("moby velocity (m/s)", [format(body[0], '.3f'), format(body[1], '.3f'), format(body[2], '.3f')])

### stop motion

In [ ]:
moby.stop_motion()

### go straight
-> set all rotation motors to zero deg

In [4]:
moby.go_straight()

In [5]:
moby.stop_motion()

### set rotation angle of each wheel(deg)

In [36]:
fl = 0
fr = 0
bl = 0
br = 0

moby.move_rotation_deg(fl=fl, fr=fr, bl=bl, br=br)

In [16]:
moby.stop_motion()

### set driving speed of each wheel(m/s)

In [17]:
fl = 0.2
fr = 0.2
bl = 0.2
br = 0.2
moby.move_driving_mps(fl=fl, fr=fr, bl=bl, br=br)

In [18]:
moby.stop_motion()

### set velocity of moby(m/s)

In [45]:
moby.set_target_vel(vx=0.2, vy=0.0 ,vw=0.0)

In [47]:
moby.stop_motion()

## Keyboard control

install python module
- pip install keyboard
- pip install ipython

|　         |**(8:+vx)**|　         |
|:---------:|:---------:|:---------:|
|**(4:+vy)**|**(5:-vx)**|**(6:-vy)**|
|**(1:-vw)**|　         |**(3:+vw)**|

 **(7:stop)**----------------**(q:quit)**

In [6]:
import keyboard
import time
from IPython.display import clear_output

vel_x = 0
vel_y = 0
vel_w = 0
vel_interval = 0.05

print("[vx, vy, vw]")
while True:
    clear_output(wait=True)
    time.sleep(0.1)
#     time.sleep(0.01)
    key = keyboard.read_key()
    if key == "q":
        print("q")
        break
    else:
        if key=="4":
            vel_y += vel_interval
        elif key=="6":
            vel_y -= vel_interval
        elif key=="8":
            vel_x += vel_interval           
        elif key=="5":
            vel_x -= vel_interval
        elif key=="3":
            vel_w += vel_interval
        elif key=="1":
            vel_w -= vel_interval
        elif key=="7":
            vel_x = 0
            vel_y = 0
            vel_w = 0
        print("[{:.6f}, {:.6f}, {:.6f}]".format(vel_x,vel_y,vel_w))
        moby.set_target_vel(vel_x, vel_y, vel_w)
        key = False

In [20]:
moby.stop_motion()

### IO Control

In [154]:
moby.turn_light(True)
# moby.turn_light(False)

In [155]:
moby.turn_buzzer(True)
# moby.turn_buzzer(False)

In [8]:
moby.pause_bumper(True)
# moby.pause_bumper(False)

In [62]:
moby.recover()

## Realtime Logger for swerving moby

dir : ~/release/IndyDeployment/MobyRTLog

q : angle

- 0 : record time

rotation motor (fl, fr, bl, br)

- 1, 2, 3, 4 (q)

- 5, 6, 7, 8 (qdot)

- 9, 10, 11, 12 (tau)

- 13, 14, 15, 16 (target q)



- 17, 18, 19, 20 (q_desired)

- 21, 22, 23, 24 (qdot_desired)

- 25, 26, 27, 28 (qddot_desired)



driving wheel (fl, fr, bl, br)

- 29, 30, 31, 32 (qdot)

- 33, 34, 35, 36 (target qdot)


In [60]:
moby.start_rt_logging()

In [63]:
moby.end_rt_logging()